# Graph Analyses
Here, we'll perform various analysis by constructing graphs and measure properties of those graphs to learn more about the data

In [56]:
import csv
from scipy.spatial import Delaunay
import numpy as np
import math

In [57]:
# Read in the data
data = open('../data/data.csv', 'r').readlines()
fieldnames = ['x', 'y', 'z', 'unmasked', 'synapses']
reader = csv.reader(data)
reader.next()

rows = [[int(col) for col in row] for row in reader]

# These will come in handy later
sorted_x = sorted(list(set([r[0] for r in rows])))
sorted_y = sorted(list(set([r[1] for r in rows])))
sorted_z = sorted(list(set([r[2] for r in rows])))

We'll start with just looking at analysis in euclidian space, then thinking about weighing by synaptic density later. Since we hypothesize that our data will show that tissue varies as we move down the y-axis (z-axis in brain) through cortical layers, an interesting thing to do would be compare properties of the graphs on each layer (ie how does graph connectivity vary as we move through layers).

Let's start by triangulating our data. We'll use Delaunay on each y layer first. Putting our data in the right format for doing graph analysis...

In [58]:
a = np.array(rows)
b = np.delete(a, np.s_[3::],1)

# Separate layers - have to do some wonky stuff to get this to work
b = sorted(b, key=lambda e: e[1])
b = np.array([v.tolist() for v in b])
b = np.split(b, np.where(np.diff(b[:,1]))[0]+1)

Now that our data is in the right format, we'll create 52 delaunay graphs

In [59]:
graphs = []
for layer in b:
    centroids = np.array(layer)
    # get rid of the y value - not relevant anymore
    centroids = np.delete(centroids, 1, 1)
    graph = Delaunay(centroids)
    graphs.append(graph)

Great - we have a list of 52 delaunay graphs. We can now perform analyses on these graphs. A simple but useful metric would be to analyze edge length distributions in each layer.

We're going to need a method to get edge lengths from 2D centroid pairs

In [60]:
def get_d_edge_length(edge):
    (x1, y1), (x2, y2) = edge
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [83]:
edge_length_list = [[]]
tri_area_list = [[]]

for del_graph in graphs:
    
    tri_areas = []
    edge_lengths = []
    triangles = []

    for t in centroids[del_graph.simplices]:
        triangles.append(t)
        a, b, c = [tuple(map(int,list(v))) for v in t]
        edge_lengths.append(get_d_edge_length((a,b)))
        edge_lengths.append(get_d_edge_length((a,c)))
        edge_lengths.append(get_d_edge_length((b,c)))
        try:
            tri_areas.append(float(Triangle(a,b,c).area))
        except:
            continue
    edge_length_list.append(edge_lengths)
    tri_area_list.append(tri_areas)

In [84]:
# edge_length_list[3]
# tri_area_list[3]
# triangles

[array([[ 58, 832],
        [ 19, 943],
        [ 19, 832]]), array([[ 19, 943],
        [ 58, 832],
        [ 58, 943]]), array([[ 97, 721],
        [ 58, 832],
        [ 58, 721]]), array([[ 58, 832],
        [ 97, 721],
        [ 97, 832]]), array([[ 19, 721],
        [ 58, 832],
        [ 19, 832]]), array([[ 58, 832],
        [ 19, 721],
        [ 58, 721]]), array([[ 58, 832],
        [ 97, 943],
        [ 58, 943]]), array([[ 97, 943],
        [ 58, 832],
        [ 97, 832]]), array([[ 58,  55],
        [ 19, 166],
        [ 19,  55]]), array([[ 19, 166],
        [ 58,  55],
        [ 58, 166]]), array([[ 19, 721],
        [ 58, 610],
        [ 58, 721]]), array([[ 58, 610],
        [ 19, 721],
        [ 19, 610]]), array([[214, 388],
        [214, 499],
        [175, 499]]), array([[175, 388],
        [214, 388],
        [175, 499]]), array([[214, 388],
        [175, 388],
        [175, 277]]), array([[214, 277],
        [214, 388],
        [175, 277]]), array([[ 19, 499],
    

In [51]:
# Note for future
del_features['d_edge_length_mean'] = np.mean(edge_lengths)
del_features['d_edge_length_std'] = np.std(edge_lengths)
del_features['d_edge_length_skew'] = scipy.stats.skew(edge_lengths)
del_features['d_edge_length_kurtosis'] = scipy.stats.kurtosis(edge_lengths)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'del_features' is not defined